## Obtain the compact csv file of USGS dataset

In [1]:
# python packages used in the sample codes

import pandas as pd
import os
import numpy as np
import csv
import glob
import codecs
import cirpy

#### Download spectrum data and CAS registry number and transform to SMILES

In [2]:
input_file = sorted(glob.glob("usgs_splib07/ASCIIdata/ASCIIdata_splib07a/ChapterO_OrganicCompounds/*AREF.txt"))
smiles_list = []
spectrum_list = []
smiles_error_list = []

for i, file_name in enumerate(input_file):
    file_html = file_name.replace('txt', 'html')\
                         .replace('ASCIIdata_splib07a/ChapterO_OrganicCompounds', '')\
                         .replace('splib07a_', '')\
                         .replace('ASCIIdata/', 'HTMLmetadata')
    with codecs.open(file_html, 'r', 'utf-8', 'ignore') as fileobj:
        lines = fileobj.readlines()
        line_cas = [line for line in lines if 'CAS' in line]
        cas = ''.join(line_cas)                   
        cas = cas[7:]
        if line_cas!=[]:
            to_smiles = cirpy.resolve(cas, "smiles")
            if to_smiles == None: 
                smiles_error_list.append(file_html)
            elif to_smiles != None:
                smiles_list = np.append(smiles_list,to_smiles)
                spectrum_data = pd.read_table(file_name)
                spectrum_data = spectrum_data.astype('float32')
                spectrum_list = np.append(spectrum_list,spectrum_data)

#### Remove duplicate data and converting reflectance to absorbance

In [3]:
spectrum_list = spectrum_list.reshape(len(smiles_list),-1)
spectrum_list = np.array(spectrum_list, dtype='float32')

duplicate_list = [1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 21, 26, 28, 31, 32,\
                  33, 36, 37, 40, 42, 43, 48, 53, 55, 57, 59, 61, 63, 66,\
                  68, 70, 73, 75, 77, 79, 80, 82, 83, 84, 86, 87, 93, 96,\
                  97, 98, 104, 105, 106, 110, 111, 113, 117]
smiles_list_nodupli = np.delete(smiles_list,[duplicate_list])
spectrum_list_nodupli = np.delete(spectrum_list,[duplicate_list],0)
spectrum_list_nodupli = 2 - np.log10(spectrum_list_nodupli)
spectrum_list_nodupli = np.array(spectrum_list_nodupli, dtype='float32')

#### Make Dataset_USGS.csv

In [4]:
df = pd.DataFrame(spectrum_list_nodupli)
df.insert(0, 'SMILES', smiles_list_nodupli)
df = df.sample(frac=1,random_state=1)
df.to_csv('Dataset_USGS.csv')